In [8]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/My Drive/Image Anomaly')

ls

curr_dir = os.getcwd()

#Training the model
save_model = 'anomaly_detector.model'
dataset = curr_dir+'/train'
print(dataset)

from pyimagesearch.features import load_dataset
from sklearn.ensemble import IsolationForest
import argparse
import pickle

# load and quantify our image dataset
print("[INFO] preparing dataset...")
data = load_dataset(dataset, bins=(3, 3, 3))
# train the anomaly detection model
print("[INFO] fitting anomaly detection model...")
model = IsolationForest(n_estimators=100, contamination=0.01,
	random_state=42)
#data = data.reshape(-1, 1)
model.fit(data)

f = open(save_model, "wb")
f.write(pickle.dumps(model))
f.close()

#Finding Anomaly
model = 'anomaly_detector.model'
image_path = curr_dir+'/examples/2.PNG'
print(image_path)

from pyimagesearch.features import quantify_image
import argparse
import pickle
import cv2
from google.colab.patches import cv2_imshow

# load the anomaly detection model
print("loading anomaly detection model...")
model_path = open('anomaly_detector.model', 'rb')
model = pickle.loads(model_path.read())
# load the input image, convert it to the HSV color space, and
# quantify the image as we did during training

image = cv2.imread(image_path)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
features = quantify_image(hsv, bins=(3, 3, 3))

# use the anomaly detector model and extracted features to determine
# if the example image is an anomaly or not

preds = model.predict([features])[0]
label = "anomaly" if preds == -1 else "normal"
color = (0, 0, 255) if preds == -1 else (0, 255, 0)
# draw the predicted label text on the original image
cv2.putText(image, label, (10,  25), cv2.FONT_HERSHEY_SIMPLEX,
  0.7, color, 2)
# display the image
cv2_imshow(image)
cv2.waitKey(0)